In [ ]:
!pip install boto3 langchain langchain_community -U

In [1]:
import json
import boto3
import random
import pandas as pd

In [2]:
bedrock_runtime = boto3.client('bedrock-runtime')

In [3]:
filename = 'data/场景2-样本数据-2.xlsx'
df = pd.read_excel(filename,nrows=2000)
df.head()

,标题,url,摘要,网站,作者,发布时间,是否推送
0,在AI时代，识别专属于“你”的骗局,https://www.toutiao.com/article/73338475961945...,结合“平安包头”官微等渠道披露的案件细节，《瞭望》提到的受害人可能是福州市一家科技公司的法人...,今日头条-社会,袁文泽,20240210140858,不推送
1,排队发红包了\n[爆竹][爆竹][爆竹]\n两个小孩压岁钱比我一个月工资还多\n到哪说理去\...,https://www.iesdouyin.com/share/video/73338534...,排队发红包了[爆竹][爆竹][爆竹]两个小孩压岁钱比我一个月工资还多到哪说理去😅😅😅,抖音,蓝梦雨,20240210141731,不推送
2,//@小林是李昀锐:#致1999年的自己#新岁已至，幸福常伴，天长地“九”！,https://weibo.com/7616066284/NFU8pzv7c,//@小林是李昀锐:#致1999年的自己#新岁已至，幸福常伴，天长地“九”！,新浪微博,泪下寻深冬,20240210141840,不推送
3,看看，这就是不给钱的下场，直接到他家门口堵,https://www.iesdouyin.com/share/video/73338543...,看看，这就是不给钱的下场，直接到他家门口堵,抖音,来自星星的你,20240210142056,推送
4,哈哈哈，新年就是要玩得开心‪！我也来放烟花了[馋嘴],https://weibo.com/5972629295/NFU994YGC,哈哈哈，新年就是要玩得开心‪！我也来放烟花了[馋嘴],新浪微博,挚爱月与歌,20240210142029,不推送


In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df_news = df.apply(lambda x :"{"+f'"title":{x["标题"]},"abstract":{x["摘要"]},"news_id":{x["发布时间"]}'+"}" ,axis=1)
df_news

0       {"title":在AI时代，识别专属于“你”的骗局,"abstract":结合“平安包头”...
1       {"title":排队发红包了\n[爆竹][爆竹][爆竹]\n两个小孩压岁钱比我一个月工资还...
2       {"title"://@小林是李昀锐:#致1999年的自己#新岁已至，幸福常伴，天长地“九”...
3       {"title":看看，这就是不给钱的下场，直接到他家门口堵,"abstract":看看，这...
4       {"title":哈哈哈，新年就是要玩得开心‪！我也来放烟花了[馋嘴],"abstract"...
                              ...                        
1995    {"title":#龙年第一条微博#\n潜龙腾跃而起 便是人间春色万里[送福] ​​,"ab...
1996    {"title":我家狗看见这些烟花就会尖叫着跑上前去，吓死我了,"abstract":我家...
1997    {"title":借着烟花许愿 希望我们都能得偿所愿，2024龙年新春大吉#欢欢喜喜过大年 ...
1998    {"title":#烟花 #大洋湾新年有好戏 排队两个多小时，就为了25分钟的烟花秀，值了,...
1999    {"title":我的宝😍😍😍#我家的小暖男正在渐渐长大 #小神兽在身边 #一转眼都已经这么...
Length: 2000, dtype: object

In [6]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate

In [7]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.1,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0.1,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [8]:
template_extract = \
"""You are a great text classifer, and now you are asked to analyze the news from internet and extract and classify the news which are fallen into categories:
[官员舆情,司法舆情,民族舆情,暴恐舆情,经济舆情,旅游舆情,民生舆情,环境舆情,教育舆情,文化舆情,宗教舆情,自然灾害,敏感舆情,医疗舆情,人为灾害]
Below is the definition of each category:
官员舆情- 新闻内容涉及：独断专行,履历造假,工作作风,官商勾结,挥霍性腐败,挪用公款,公车私用,招兵黑幕,滥用职权,工作失职,官职买卖
司法舆情- 新闻内容涉及：目无法纪,伪证假证,冤假错案,警民冲突,暴力执法,选举换届
民族舆情 -新闻内容涉及：民族政策,民族习俗,传统节日,民族信仰
暴恐舆情 - 新闻内容涉及：恐怖组织,自杀袭击,生化袭击,独狼式袭击
经济舆情 - 新闻内容涉及：偷税漏税,非法生产,非法买卖,走私、造假,非法传销,证件票券犯罪,扰乱市场秩序
旅游舆情 - 新闻内容涉及：旅游管理,旅游投诉,非法宰客,景区门票,景点设施,虚假景点
民生舆情 - 新闻内容涉及：房屋交易,违规建筑,农资质量,食品安全,豆腐渣工程,欠薪讨薪,传染病疫情,婚育婴、养老,就业、失业、职业疾病,土地、拆迁、验收纠纷
环境舆情 - 新闻内容涉及：生物入侵，土地污染，资源乱采，生物多样性破坏，大气污染，水体污染，水土流失，盗猎盗采
教育舆情 - 新闻内容涉及：学生学籍，校园欺凌，校园暴力，校园安全，违规办学，教育政策，减招问题，网瘾问题，幼儿性侵，试题泄露，教育收费，学校体罚，非法补课，校风师德，校园环境，校园设施，学术造假
文化舆情 - 新闻内容涉及：文物保护，文化政策，文化补贴，晚会演出，非物质文化遗产保护，文物展览，社会思潮，文化发展，电视电台
宗教舆情 - 新闻内容涉及：非法传教，宗教信仰，邪教传播，宗教场所
自然灾害 - 新闻内容涉及：洪水灾害，风灾灾害，海洋灾害，大雾灾害，高温旱灾，雨雪灾害，冻害灾害，生物灾害，地质灾害，泥石流灾害
敏感舆情 - 新闻内容涉及：后台关系，面子工程，贪污受贿，暗箱操作，霸占问题，拐卖、盗窃，涉赌、涉黄，扰乱公共秩序，国有资产流失，抢劫、毁坏公物，危险品、涉爆、投毒，黑恶势力，绑架勒索，妨害公务，城管执法，执法不当，人身侵犯，游行示威，围堵机关，上访/抗议，自杀/自焚，暴力/涉枪
医疗舆情 - 新闻内容涉及：医疗事故，医患关系，医疗改革，医疗设施，医疗卫生，医疗政策，违规办院，药品价格，医疗报销，挂号问题，票贩问题，患者体验，医闹纠纷，医德医风，医生技术，假药黑药
人为灾害 - 新闻内容涉及：公路事故，水上事故，矿难事故，铁路事故，溺水事故，火灾事故，空难事故，核辐射事故，水电气事故，公共设施事故
其他舆情 - 新闻内容涉及：其他不在以上的分类中内容

Here is the tasks:
Only find the quotes from the message that are are fallen into categories and classify them, and then print them in xml tag <response>. 
Quotes should be exactly the same as original text.
If there are no relevant quotes at all, write "No relevant quotes" instead.

Here is a chat history you will analyze
<doc>
{context}
</doc>


please enclose your analysis results in xml tag <response>.

the output format is json, for example:
<response>
[
{{"title":xxx,"abstract":xxx,"url":xxx,"category":xxx}}
]
</response>


Skip the preamble, go straight into the answer.
"""

In [9]:
template_sentiment = \
"""You are a text classifier
Give you a list of news from internet you will classify them to the predefined categories [官员舆情,司法舆情,民族舆情,暴恐舆情,经济舆情,旅游舆情,民生舆情,环境舆情,教育舆情,文化舆情,宗教舆情,自然灾害,敏感舆情,医疗舆情,人为灾害,其他],
output the category and a confidence score between 0 to 10, that represents how relevant it is to the category, 10 means most relevant, and 0 is not relevant. 

Below is the definition of each category:
官员舆情- 新闻内容涉及：独断专行,履历造假,工作作风,官商勾结,挥霍性腐败,挪用公款,公车私用,招兵黑幕,滥用职权,工作失职,官职买卖
司法舆情- 新闻内容涉及：目无法纪,伪证假证,冤假错案,警民冲突,暴力执法,选举换届
民族舆情 -新闻内容涉及：民族政策,民族习俗,传统节日,民族信仰
暴恐舆情 - 新闻内容涉及：恐怖组织,自杀袭击,生化袭击,独狼式袭击
经济舆情 - 新闻内容涉及：偷税漏税,非法生产,非法买卖,走私、造假,非法传销,证件票券犯罪,扰乱市场秩序
旅游舆情 - 新闻内容涉及：旅游管理,旅游投诉,非法宰客,景区门票,景点设施,虚假景点
民生舆情 - 新闻内容涉及：房屋交易,违规建筑,农资质量,食品安全,豆腐渣工程,欠薪讨薪,传染病疫情,婚育婴、养老,就业、失业、职业疾病,土地、拆迁、验收纠纷
环境舆情 - 新闻内容涉及：生物入侵，土地污染，资源乱采，生物多样性破坏，大气污染，水体污染，水土流失，盗猎盗采
教育舆情 - 新闻内容涉及：学生学籍，校园欺凌，校园暴力，校园安全，违规办学，教育政策，减招问题，网瘾问题，幼儿性侵，试题泄露，教育收费，学校体罚，非法补课，校风师德，校园环境，校园设施，学术造假
文化舆情 - 新闻内容涉及：文物保护，文化政策，文化补贴，晚会演出，非物质文化遗产保护，文物展览，社会思潮，文化发展，电视电台
宗教舆情 - 新闻内容涉及：非法传教，宗教信仰，邪教传播，宗教场所
自然灾害 - 新闻内容涉及：洪水灾害，风灾灾害，海洋灾害，大雾灾害，高温旱灾，雨雪灾害，冻害灾害，生物灾害，地质灾害，泥石流灾害
敏感舆情 - 新闻内容涉及：后台关系，面子工程，贪污受贿，暗箱操作，霸占问题，拐卖、盗窃，涉赌、涉黄，扰乱公共秩序，国有资产流失，抢劫、毁坏公物，危险品、涉爆、投毒，黑恶势力，绑架勒索，妨害公务，城管执法，执法不当，人身侵犯，游行示威，围堵机关，上访/抗议，自杀/自焚，暴力/涉枪
医疗舆情 - 新闻内容涉及：医疗事故，医患关系，医疗改革，医疗设施，医疗卫生，医疗政策，违规办院，药品价格，医疗报销，挂号问题，票贩问题，患者体验，医闹纠纷，医德医风，医生技术，假药黑药
人为灾害 - 新闻内容涉及：公路事故，水上事故，矿难事故，铁路事故，溺水事故，火灾事故，空难事故，核辐射事故，水电气事故，公共设施事故
其他舆情 - 新闻内容涉及：其他不在以上的分类中内容


Here are the news:
<news>
{relevant_info}
</news>

Please follow below requirements:
1. You will strictly be based on the document in <news>.
2. please enclose your analysis results in xml tag <sentiment>.
3. output a score between 0 -10,

the output format is json for example:
<sentiment>
[
{{"title":xxx,"abstract":xxx,"category":xxx,"url":xxx,"score":xx}}
]
</sentiment>

Skip the preamble, go straight into the answer. 

"""

In [10]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<response>(.*?)</response>"
        match = re.search(pattern, content, re.DOTALL)
        if match:
            text = match.group(1)
            text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
            return text
        else:
            return 'No relevant quotes'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"


class CustOuputParser2(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<sentiment>(.*?)</sentiment>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'No sentiment'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    

In [11]:
def split_into_chunks(arr, n):
    chunks = []
    for i in range(0, len(arr), n):
        chunks.append(arr[i:i+n])
    return chunks

In [12]:
chunks = split_into_chunks(df_news.values,250)

In [13]:
print(f"total chunks {len(chunks)}")

total chunks 8


In [14]:
text =  "\n".join(chunks[0])
print(len(text))
# print(text)

38665


In [15]:
# from langchain_core.output_parsers import StrOutputParser
# prompt = ChatPromptTemplate.from_template(template)
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_sentiment = ChatPromptTemplate.from_template(template_sentiment)

In [16]:
output_parser = CustOuputParser()
output2_parser  = CustOuputParser2()

llm = llm_haiku
chain_1 = prompt_extract | llm |output_parser
chain_2 = prompt_sentiment | llm|output2_parser
# chain_3 = ({'relevant_info':chain_1,'topic':itemgetter('topic')})|prompt_sentiment | llm_haiku|output2_parser

In [17]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
## 如果没有相关内容则无须invoke chain2
def route(info):
    if not 'no relevant quotes' in info['relevant_info'].lower():
        return chain_2

In [18]:
##测试加入了路由选择的chainfull
full_chain = ({'relevant_info':chain_1,})| RunnableLambda(route)
answer = full_chain.invoke({'context':text})

<response>
[
{"title":"在AI时代，识别专属于"你"的骗局","abstract":"结合"平安包头"官微等渠道披露的案件细节，《瞭望》提到的受害人可能是福州市一家科技公司的法人代表郭先生...去年4月，他10分钟被骗430万元的案子一度成为热搜...","category":"经济舆情"},
{"title":"排队发红包了","abstract":"排队发红包了[爆竹][爆竹][爆竹]两个小孩压岁钱比我一个月工资还多到哪说理去😅😅😅","category":"民生舆情"},
{"title":"//@小林是李昀锐:#致1999年的自己#新岁已至，幸福常伴，天长地"九"！","abstract":"//@小林是李昀锐:#致1999年的自己#新岁已至，幸福常伴，天长地"九"！","category":"文化舆情"},
{"title":"看看，这就是不给钱的下场，直接到他家门口堵","abstract":"看看，这就是不给钱的下场，直接到他家门口堵","category":"民生舆情"},
{"title":"哈哈哈，新年就是要玩得开心‪！我也来放烟花了[馋嘴]","abstract":"哈哈哈，新年就是要玩得开心‪！我也来放烟花了[馋嘴]","category":"旅游舆情"},
{"title":"今天2024年🌟，新的一年里Ssp护工祝福大家🎵✨✨🌵 一帆风顺⛵🎏✨✨🌷 二龙腾飞✈🚀✨✨🌺 三羊开泰🌅🎆✨✨🍀 四季平安🌈🎑✨✨🌻 五福临门😄💪✨✨🌸 六六大顺🎉","abstract":"临门😄💪✨✨🌻 六六大顺🎉🎯✨✨🌹 七星高照🙏🎇✨✨💐 八方来财🏧💰✨✨🌷 九九同心💏🎎✨✨☀ 十全十美🙌🈵✨✨🌺 百事亨通👏🎰✨✨🌺 千事吉祥🉐☀✨✨🍀 万事如意🎐👍✨✨好运连连[福][烟花....","category":"民生舆情"},
{"title":"寻人启事 丁样香，绰名张红，身高1米5，体重180 斤，身穿黑色到膝盖下左右衣服，衣服带 帽子，黑色裤子，黑色面包鞋，带一个双 肩黑兰色背包，户籍江苏省东台市许可镇 西灶村三组，于20","abstract":"寻人启事 丁样香，绰名张红，身高1米5，体重180 斤，身穿黑色到膝盖下左右衣服，衣服带 帽子，黑色裤子，黑色面包鞋，带一个双 肩黑兰色背包，户籍江苏省东台市许可镇 西灶村三组，于2024

In [ ]:
answer

In [ ]:
# question = "is there any content relevant to auction house?"

#'please list all the content if it is relevant and classify the sentiment of each content into [positive,neutral,negative]'

# answer = chain_2.invoke({'topic':"auction house",
#                        'context':text})

In [ ]:
# print(answer)

In [ ]:
def extract_text_and_sentiment(text) -> list:
    pattern = r'^(.*?)\s*\[(.*?)\]'
    # pattern = r'"(.*?)"\s*\[(.*?)\]$'
    if not text:
        return []
    result = []
    for line in text.split('\n'):
        match = re.match(pattern, line)
        if match:
            # print(match)
            text_part = match.group(1)
            quoted_texts = re.findall(r'"(.*?)"', text_part)
            # print(quoted_texts)
            sentiment_part = match.group(2)
            result.append((quoted_texts[0], sentiment_part))
    return result

In [ ]:
extract_text_and_sentiment(answer)

In [ ]:
#save a list to a local file, if the file is exist then append the list to the file
def save_list(list,file_name):
    with open(file_name,'a') as f:
        for item in list:
            f.write("%s\n" % item)
    f.close()
    return True

In [ ]:
#save a list to a csv file using pandas, if the file is exist then append the list to the file
import pandas as pd
import os
def save_to_csv(data, filename):
    new_data = pd.DataFrame(data, columns=['text','sentiment'])
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        #append data to df
        df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data
    df.to_csv(filename, index=False)
    return df

循环每个chunk进行输出,每次追加到csv文件保存，

In [ ]:
import time
all_result = []
filename = 'parsed_result'

##断点跳过
skip_num =238
for i,chunk in enumerate(chunks[:1000]):
    if i < skip_num:
        continue
    t1 = time.time()
    print(f'--------chunk idx:{i}-------')
    text =  "\n".join(chunk)
    answer = full_chain.invoke({'topic':"auction house",
                       'context':text})
    extract_ret = extract_text_and_sentiment(answer)
    print(f'\nextract_ret:\n{extract_ret}')
    time.sleep(2)
    print(f'---time cost {time.time()-t1} s -----\nsleep 10s for token tpm\n\n')
    if extract_ret:
        all_result+=extract_ret
        save_to_csv(extract_ret,filename+f"chunk_{i}.csv")
        save_to_csv(extract_ret,filename+f"_all.csv")

In [ ]:
len(all_result)